In [2]:
# code and data: https://github.com/rpeden/cat-or-not/releases
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from PIL import Image
from random import shuffle, choice
import numpy as np
import os

In [3]:
IMAGE_SIZE = 256
IMAGE_DIRECTORY = './data/training_set'

In [4]:
def label_img(name):
    if name == 'cats': return np.array([1, 0])
    elif name == 'notcats' : return np.array([0, 1])

In [5]:
def load_data():
    print("Loading images...")
    train_data = []
    directories = next(os.walk(IMAGE_DIRECTORY))[1]

    for dirname in directories:
        print("Loading {0}".format(dirname))
        file_names = next(os.walk(os.path.join(IMAGE_DIRECTORY, dirname)))[2]
        for i in range(len(file_names)): # len(file_names)
            image_name = choice(file_names)
            image_path = os.path.join(IMAGE_DIRECTORY, dirname, image_name)
            label = label_img(dirname)
            if "DS_Store" not in image_path:
                img = Image.open(image_path)
                img = img.convert('L')
                img = img.resize((IMAGE_SIZE, IMAGE_SIZE), Image.LANCZOS)
                train_data.append([np.array(img)/255, label])
    return train_data

In [5]:
training_data = load_data()
training_images = np.array([i[0] for i in training_data]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE)
training_labels = np.array([i[1] for i in training_data])

Loading images...
Loading cats
Loading notcats


In [6]:
def create_model():
    model = Sequential([
        tf.keras.layers.Input((IMAGE_SIZE, IMAGE_SIZE)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    return model

In [11]:
print('creating model')
model = create_model()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print('training model')
model.fit(training_images, training_labels, batch_size=64, epochs=5)

creating model
training model
Epoch 1/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 7s 69ms/step - accuracy: 0.6833 - loss: 5.9553
Epoch 2/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - accuracy: 0.7065 - loss: 0.5864
Epoch 3/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.7049 - loss: 0.5794
Epoch 4/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 6s 70ms/step - accuracy: 0.7123 - loss: 0.5649
Epoch 5/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.7062 - loss: 0.5823


In [12]:
IMAGE_DIRECTORY = './data/test_set'
test_data = load_data()
test_images = np.array([i[0] for i in test_data]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE)
test_labels = np.array([i[1] for i in test_data])

Loading images...
Loading cats
Loading notcats


In [16]:
loss, acc = model.evaluate(test_images, test_labels, verbose=1)
print(f"accuracy: {acc*100:.2f}%")

84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7323 - loss: 0.5588
accuracy: 37.58%
